In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

create or replace temporary view control_src as
select
  'lakefed_ingest' as job_name,
  'sf_1000_liquid' as task_collection,
  'delta' as src_type,
  table_catalog as src_catalog,
  table_schema as src_schema,
  table_name as src_table,
  'lakefed_ingest' as sink_catalog,
  'tpcds_sf1000' as sink_schema,
  table_name as sink_table,
  null as sink_cluster_cols,
  'full' as load_type,
  false as load_partitioned,
  '*' as select_list,
  null as watermark_col_name,
  null as watermark_col_type,
  null as watermark_col_start_value,
  null as partition_col,
  null as partition_size_mb,
  true as task_enabled
from tpcds.information_schema.tables
where table_schema = 'sf_1000_liquid'
  and table_catalog = 'tpcds';

In [0]:
merge with schema evolution into identifier(:table) as t
using control_src as s
on t.src_catalog = s.src_catalog
and t.src_schema = s.src_schema
and t.src_table = s.src_table
when matched
  then update set *
when not matched
  then insert *

In [0]:
-- Set store_sales to partitioned ingest
update identifier(:table)
set 
  task_collection = 'store_sales',
  sink_cluster_cols = array('ss_sold_date_sk'),
  load_partitioned = true,
  partition_col = 'ss_sold_date_sk',
  partition_size_mb = 256
where
  sink_catalog = 'lakefed_ingest'
  and sink_schema = 'tpcds_sf1000'
  and sink_table = 'store_sales'